Transform: Split and create a dimenstion table for unique questions(question asked for Pregnancy Risks Assessment ) from CSV data. There are 520K records, show a report on number of  unique questions.

Create a new Gold Table to keep unique questions and publish

|Colmuns| DATA Type
|--|--|
 question_no| INT
  Question_Id | STRING
  Question | STRING



###STEP1: 
|A) CREATE A STAGING TABLE TO PULL DISTINCT QUESTIONS FROM MAIN CSV RECORD|
|--|
B)DATA CLEANUP WITH AMBIGUOUS QUESTIONS (SAME QUESTION MULTIPLE TIME DUE TO ADDITIONAL CHARACTER LIKE COMMA)|
C)INSERT CLEANED UP DATA TO GOLD TABLE WITH UNQIUE QUESTIONS

In [0]:
/*Create a staging table into Silver schema*/
USE CATALOG materncare_anlytics;
CREATE OR REPLACE TABLE silver.prams_questions_dim_stage_2
(
  questionno INTEGER,
  QuestionId string,
  Question string
)

In [0]:
/*Insert distinct question (256 questions)*/
INSERT INTO materncare_anlytics.silver.prams_questions_dim_stage_2
SELECT DISTINCT
  SUBSTRING(questionid, 4) as questionno,
  QuestionId,
  Question 
FROM materncare_anlytics.silver.prams_data
ORDER BY questionno ASC


 


b)DATA CLEANUP WITH AMBIGUOUS QUESTIONS (SAME QUESTION MULTIPLE TIME DUE TO ADDITIONA CHARACTER LIKE COMMA)

In [0]:
/*Create another staging table with ambiguous question numbers*/
USE CATALOG materncare_anlytics;
CREATE OR REPLACE TABLE silver.prams_questions_dim_stage_1
(
  questionno INTEGER,
  QuestionId string,
  Question string,
  recordno INTEGER
)

In [0]:
/*Insert recoirds with additional column partioning same record with count*/
INSERT INTO materncare_anlytics.silver.prams_questions_dim_stage_1
SELECT 
  *,
  CASE 
    WHEN COUNT(*) OVER (PARTITION BY questionno) > 1 THEN true
    ELSE false
  END --AS has_duplicate_questionno
FROM materncare_anlytics.silver.prams_questions_dim_stage_2
ORDER BY questionno ASC


/*reInsert recoirds with additional column partioning same record with count*/
 INSERT INTO materncare_anlytics.silver.prams_questions_dim_stage_2
 SELECT
      questionno,
      QuestionId,
      Question,
      ROW_NUMBER() OVER (PARTITION BY questionno ORDER BY QuestionId ASC) AS rn
    FROM materncare_anlytics.silver.prams_questions_dim_stage_1

In [0]:
/*Delete ambiguios questions from staging 2 table*/
DELETE from materncare_anlytics.silver.prams_questions_dim_stage_2
where TRIM(question) IN (
SELECT A.Question
FROM materncare_anlytics.silver.prams_questions_dim_stage_2 A
JOIN materncare_anlytics.silver.prams_questions_dim_stage_1 B
ON TRIM(A.question) = TRIM(B.question)
WHERE B.recordno = 1)
    

In [0]:
SELECT 
ROW_NUMBER() OVER (ORDER BY questionno, QuestionId ASC) AS row_num,
questionno,
QuestionId,
Question
FROM materncare_anlytics.silver.prams_questions_dim_stage_2

MOVE TRANSFORMED DATA TO GOLD TABLE

In [0]:
USE CATALOG materncare_anlytics;
CREATE SCHEMA IF NOT EXISTS gold;
CREATE OR REPLACE TABLE gold.prams_questions_dim
(
  row_num INT,
  question_no INT,
  Question_Id STRING,
  Question STRING
)

In [0]:
INSERT INTO materncare_anlytics.gold.prams_questions_dim
SELECT 
ROW_NUMBER() OVER (ORDER BY questionno, QuestionId ASC) AS row_num,
questionno,
QuestionId,
Question
FROM materncare_anlytics.silver.prams_questions_dim_stage_2

Drop staging tables

In [0]:
DROP TABLE silver.prams_questions_dim_stage_1;
DROP TABLE silver.prams_questions_dim_stage_2